# Brain Imaging Data Structure (BIDS) data conversion tutorial 

In the following Jupyter notebook, we will introduce you to BIDS basics. You will download some phantom data in neuroimaging formats as they come of a scanner and then work on converting it to BIDS format.
Afterwards we will also spend time on how we can utilize data in BIDS format for easy processing of neuroimaging data.


## Collect the Repository and Install all Dependencies
This may take some time.

In [ ]:
!if [ -d "outreach" ]; then echo "Repo already cloned." && cd outreach && git pull && cd ..; else echo "Collecting Outreach Repository" && git clone https://github.com/openneuropet/outreach.git; fi
from os.path import basename
from os import getcwd
if basename(getcwd()) == 'PETBIDS-Onboarding2025':
    pass
else:
    %cd outreach/PETBIDS-Onboarding2025/

!pip install nipype jedi pypet2bids
!apt-get install pigz tree

# Install Deno
!curl -fsSL https://deno.land/install.sh | sh
!export DENO_INSTALL="/root/.deno"
!export PATH="$DENO_INSTALL/bin:$PATH"

## Download Phantom ZIP and Extract

In [ ]:
!if [ ! -f "PHANTOMS.zip" ]; then wget -O PHANTOMS.zip https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip; fi
# unzip quietly in either case
!unzip -q -o PHANTOMS.zip

## Install Tree

In [ ]:
import subprocess
check_for_tree = subprocess.run(['which', 'tree'], capture_output=True)
if check_for_tree.returncode == 0:
    pass
else:
    import platform
    operating_system = platform.system()
    if operating_system == 'Linux':
        subprocess.run("apt-get install tree -y", shell=True)
    elif operating_system == 'Darwin':
        subprocess.run("brew install tree", shell=True)
    else:
        print("You're on your own windows user.")

## Check for dcm2niix and install if it's not present.

Additionally, we tell pypet2bids where the dcm2niix executable is located at, this is best practice on windows and any sort of virtual environment/notebook as the $PATH variable can be "wonky" in the later case.

In [ ]:
# check for dcm2niix
import platform
import os
from pathlib import Path

check_dcm2niix = subprocess.run("which dcm2niix", shell=True, capture_output=True)
if check_dcm2niix.returncode == 0:
    print('dcm2niix is installed')
    version = subprocess.run('dcm2niix --version', shell=True, capture_output=True).stdout.decode()
    print(version)
    # set dcm2niix path as this is running in an ipython notebook
    dcm2niix_path = subprocess.run("which dcm2niix", shell=True, capture_output=True)
    subprocess.run(f"dcm2niix4pet --set-dcm2niix-path {dcm2niix_path.stdout.decode()}", shell=True)
else:
    print('dcm2niix is not installed')
    operating_system = platform.system()
    print('Your operating system is detected as '+operating_system)
    if operating_system == 'Linux':
        dcm2niix_install_dir = Path("dcm2niix_install")
        print(f"dcm2niix_install_dir {dcm2niix_install_dir}")
        if not dcm2niix_install_dir.exists():
            os.mkdir(dcm2niix_install_dir)
        subprocess.run("curl -fLO https://github.com/rordenlab/dcm2niix/releases/download/v1.0.20230411/dcm2niix_lnx.zip",
                         cwd=dcm2niix_install_dir,
                         shell=True)
        subprocess.run("unzip dcm2niix*.zip",
                         shell=True,
                         cwd=dcm2niix_install_dir)
        if str(dcm2niix_install_dir) not in os.environ.get('PATH', ''):
            os.environ['PATH'] += os.pathsep + str(dcm2niix_install_dir.resolve())
        # ensure it's on the path
        check_dcm2niix_on_path = subprocess.run('dcm2niix --version && which dcm2niix', shell=True, capture_output=True)
        check_dcm2niix_on_path.stdout.decode()
        print('dcm2niix installed successfully')
    elif operating_system == 'Darwin':
        subprocess.run("brew install dcm2niix", shell=True)
        print('dcm2niix installed successfully')
    else:
        print("You're on your own windows user.")

In [ ]:
# import the relevant Python packages
import numpy
import nibabel
import nipype
import matplotlib
import subprocess

## Setup the bids-validator
This is the easiest way to get the bids validator running on a Colab Notebook, from here on out it can be called with `bids-validator()`

In [ ]:
!npm install -g bids-validator

## Take a quick look at the raw dicom and ecat data that we've unzipped into this project folder.

In [ ]:
!tree OpenNeuroPET-Phantoms/sourcedata --filelimit 15

## Examine and convert some dicoms obtained from our PHANTOMS.zip with dcm2niix4pet.

In [ ]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder

In [ ]:
!tree ./mynewfolder

I could also add additional information regarding my data using additional flags:

In [ ]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder2 --kwargs TimeZero=ScanStart Manufacturer=Siemens ManufacturersModelName=Biograph InstitutionName="NIH Clinical Center" BodyPart=Phantom Units=Bq/mL TracerName=none TracerRadionuclide=F18 InjectedRadioactivity=81.24 SpecificRadioactivity=13019.23 ModeOfAdministration=infusion FrameTimesStart=0 AcquisitionMode="list mode" ImageDecayCorrected=true ImageDecayCorrectionTime=0 AttenuationCorrection=MR-corrected FrameDuration=300 FrameTimesStart=0

Now check if we have created a valid BIDS dataset

In [ ]:
import json
dataset = "./mynewfolder"
result = subprocess.run(
    ["bids-validator", dataset, "--json"],
    capture_output=True, text=True
)

report = json.loads(result.stdout)
print(report)

Hmm, what is wrong? Let's look at our file tree again.

In [ ]:
!tree ./mynewfolder2

Basically we have created the appropriate .nii.gz and .json files, but we haven't followed the proper BIDS naming convention!

We have done this for you here:

In [ ]:
!tree ./OpenNeuroPET-Phantoms/sub-SiemensBiographNIMH/ --filelimit 15

Now let's see if the dataset is valid....this time we will validate it differently!

## Online validation

Go to https://bids-standard.github.io/legacy-validator/ and select the folder `OpenNeuroPET-Phantoms` to see if it is valid.
(Note we are using the legacy validator since the other one has a runtime issue right now.)

You should expect to see some warnings, but the dataset itself is a valid BIDS dataset

## More advanced BIDS conversion

So even though there are libraries that do part of the BIDS conversion for you, there is often some manual work to be done wrt file renaming or adding additional files on top.

Converters like EZBids or BIDScoin try to even alleviate this burden.

We will play with EZBids!

Go to [EZBids](https://brainlife.io/ezbids/)